<a href="https://colab.research.google.com/github/AstroBoy1/santander/blob/master/vm_train_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
***

In [1]:
!pip install vecstack
!pip install catboost

In [2]:
from google.colab import drive
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn import svm
from collections import Counter
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import time
import statistics
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from vecstack import stacking
from vecstack import StackingTransformer
import pandas as pd
from glob import glob
import re 
np.random.seed(0) # ensure reproducibility
np.set_printoptions(suppress = True)
from sklearn.datasets import make_classification
from sklearn.metrics import log_loss
# Models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from keras.wrappers.scikit_learn import KerasClassifier
# Stacking
from vecstack import stacking
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization, GaussianNoise
from keras import callbacks
import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier
import warnings
warnings.simplefilter("ignore")
from keras.callbacks import EarlyStopping, TensorBoard
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
import random

Using TensorFlow backend.


In [3]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_fn = '/content/gdrive/My Drive/santander_data/train.csv'
valid_fn = '/content/gdrive/My Drive/santander_data/test.csv'
pred_fn = '/content/gdrive/My Drive/santander_data/submission12.csv'
train_data_df = pd.read_csv(train_fn)
test_data_df = pd.read_csv(valid_fn)
train_data_x = train_data_df.drop(columns=["ID_code", "target"]).values
train_data_y = train_data_df["target"].values
test_data_x = test_data_df.drop(columns=["ID_code"]).values

# Prepare data

In [0]:
n_classes = 2
length = 1000
X, y = train_data_x[:length], train_data_y[:length]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
save_directory = '/content/gdrive/My Drive/santander_results/'

# LGB, XGB, Catboost params

In [0]:
lgb_params = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1}

xgb_params = {'tree_method': 'hist',
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'learning_rate': 0.0936165921314771,
 'max_depth': 2,
 'colsample_bytree': 0.3561271102144279,
 'subsample': 0.8246604621518232,
 'min_child_weight': 53,
 'gamma': 9.943467991283027,
 'silent': 1,
}

catboost_params = {'subsample':0.36, #rawdata 0.5  ×2 0.45 ×3 0.36
                            'loss_function':'Logloss',
                           'random_strength':0,
                           'max_depth':3,
                           'eval_metric':"AUC",
                           'learning_rate':0.02,
                           #'iterations':60000,
                           'iterations':100,
                           #class_weights=[1,2],
                           'bootstrap_type':'Bernoulli',
                           #rsm=0.045,
                            'l2_leaf_reg':0.3,
                           #'task_type':"GPU",
                           'random_seed':432013,
                           'od_type':"Iter",
                           'border_count':128,
                           #'logging_level':'Silent'
                           #has_time= True 
                  }

In [0]:
cbp1 = {
    "iterations":7500,
                   "thread_count":4,
                   "loss_function":'Logloss',
                   "eval_metric":'AUC',
		           "random_seed":56498323,
                   "depth":2, #default 6
                   "learning_rate":0.04,
                   "l2_leaf_reg":2, #default 3
		           "verbose":1,
		           "use_best_model":True,
                   "od_type":'Iter',
                   "od_wait":250,
		           "nan_mode":"Min",
                   #"logging_level":'Silent',
                   #"train_dir":'train_dir'
}

cbp2 = {"loss_function":"Logloss",
                   "loss_function":'Logloss',
                   "eval_metric":'AUC',                           #"task_type":"GPU",
                           "learning_rate":0.01,
                           "iterations":10000,
                           "random_seed":42,
                           "od_type":"Iter",
                           "depth":10,
                           "early_stopping_rounds":500
       }

cbp3 = {
    "subsample":0.36, #rawdata 0.5  ×2 0.45 ×3 0.36
                            #"custom_loss":'Logloss',
                           "random_strength": 0,
                           "max_depth":3,
                   "loss_function":'Logloss',
                   "eval_metric":'AUC',                           "learning_rate":0.02,
                           "iterations":60000,
                           #class_weights=[1,2],
                           "bootstrap_type":'Bernoulli',
                           #rsm=0.045,
                           "l2_leaf_reg":0.3,
                           #"task_type":"GPU",
                           "random_seed":432013,
                           "od_type":"Iter",
                           "border_count":128
                           #has_time= True 
                          
}

cbp4 = {
    "loss_function":"Logloss",
                           "eval_metric":"AUC",
                           #"task_type":"GPU",
                           "learning_rate":0.01,
                           "iterations":70000,
                           "l2_leaf_reg":50,
                           "random_seed":432013,
                           "od_type":"Iter",
                           "depth":5,
                           "early_stopping_rounds":15000,
                           "border_count":64
                           #has_time= True 
                          
}
random_state = 432013
# https://www.kaggle.com/marcospcsj/kernel-cod-valid-cruzada-lgbm
lgbp1 = {
    'bagging_freq': 5,          'bagging_fraction': 0.335,   'boost_from_average':'false',   'boost': 'gbdt',
    'feature_fraction': 0.041,   'learning_rate': 0.0083,     'max_depth': -1,                'metric':'auc',
    'min_data_in_leaf': 80,     'min_sum_hessian_in_leaf': 10.0,'num_leaves': 13,           'num_threads': 4,
    'tree_learner': 'serial',   'objective': 'binary',      'verbosity': 1
}

# https://www.kaggle.com/bejeweled/stats-features-lgbm
lgbp2 = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : 90,
    "num_leaves" : 15,
    "learning_rate" : 0.01000123,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 150,
    "min_sum_heassian_in_leaf": 15,
    "tree_learner": "voting",
    "boost_from_average": "false",
    "lambda_l1" : 10,
    "lambda_l2" : 10,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}

# https://www.kaggle.com/sandan0o0/lgb-regression
lgbp3 = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.05,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 2019}

# https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment
lgbp4 = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    #"lambda_l1" : 5,
    #"lambda_l2" : 5,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}

# https://www.kaggle.com/indranilbhattacharya/santander-xgb-lgb-starter
xgbp1 = {'objective':"binary:logistic", 
               'eval_metric':"auc",
               'max_depth':4,
               'eta':0.05,
               'gamma':5,
               'subsample':0.7,   
               'colsample_bytree':0.7,
               'min_child_weight':50,  
               'colsample_bylevel':0.7,
               'lambda':1, 
               'alpha':0,
               'booster':"gbtree",
               'silent':0
        }

# https://www.kaggle.com/marcospcsj/simple-xgboost-test
xgbp2 = {'max_depth':3,
                   'silent':1,
                     'eval_metric': 'auc',
                   'eta':0.28071497637474263, # learning rate
                   'gamma':0,
                   'min_child_weight':0.2784483175645849,
                   'objective':'binary:logistic'
                  }

# https://www.kaggle.com/bogorodvo/starter-code-saving-and-loading-lgb-xgb-cb
xgbp3 = {'max_depth':2,  'eval_metric': 'auc',
                              'n_estimators':999999,
                              'colsample_bytree':0.3,
                              'learning_rate':0.02,
                              'objective':'binary:logistic', 
                              'n_jobs':-1
        }

# https://www.kaggle.com/silverstone1903/xgboost-baseline
xgbp4 = {'tree_method': 'hist',
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'learning_rate': 0.0936165921314771,
 'max_depth': 2,
 'colsample_bytree': 0.3561271102144279,
 'subsample': 0.8246604621518232,
 'min_child_weight': 53,
 'gamma': 9.943467991283027,
 'silent': 1}

In [0]:
xgbp_list = []
lgbp_list = []
cbp_list = []

xgbp_list.append(xgbp1)
xgbp_list.append(xgbp2)
xgbp_list.append(xgbp3)
xgbp_list.append(xgbp4)
xgbp_list.append(xgb_params)

lgbp_list.append(lgbp1)
lgbp_list.append(lgbp2)
lgbp_list.append(lgbp3)
lgbp_list.append(lgbp4)
lgbp_list.append(lgb_params)

cbp_list.append(cbp1)
cbp_list.append(cbp2)
cbp_list.append(cbp3)
cbp_list.append(cbp4)
cbp_list.append(catboost_params)

# NN

In [0]:
def auc(y_true, y_pred):
    """ROC AUC metric for both binary and multiclass classification.
    
    Parameters
    ----------
    y_true : 1d numpy array
        True class labels
    y_pred : 2d numpy array
        Predicted probabilities for each class
    """
    ohe = OneHotEncoder(sparse=False)
    y_true = ohe.fit_transform(y_true.reshape(-1, 1))
    auc_score = roc_auc_score(y_true, y_pred)
    return auc_score

# LOGGER
class Logger(callbacks.Callback):
    def __init__(self, out_path='./', patience=10, lr_patience=3, out_fn='', log_fn=''):
        self.auc = 0
        self.path = out_path
        self.fn = out_fn
        self.patience = patience
        self.lr_patience = lr_patience
        self.no_improve = 0
        self.no_improve_lr = 0

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        cv_pred = self.model.predict(self.validation_data[0], batch_size=1024)
        cv_true = self.validation_data[1]
        auc_val = roc_auc_score(cv_true, cv_pred)
        if self.auc < auc_val:
            self.no_improve = 0
            self.no_improve_lr = 0
            print("Epoch %s - best AUC: %s" % (epoch, round(auc_val, 4)))
            self.auc = auc_val
            self.model.save(self.path + self.fn, overwrite=True)
        else:
            self.no_improve += 1
            self.no_improve_lr += 1
            print("Epoch %s - current AUC: %s" % (epoch, round(auc_val, 4)))
            if self.no_improve >= self.patience:
                self.model.stop_training = True
            if self.no_improve_lr >= self.lr_patience:
                lr = float(K.get_value(self.model.optimizer.lr))
                K.set_value(self.model.optimizer.lr, 0.75*lr)
                print("Setting lr to {}".format(0.75*lr))
                self.no_improve_lr = 0

        return

# MODEL DEF
def dnn():
#     inp = Input(shape=(200, 1))
#     d1 = Dense(16, activation='relu')(inp)
#     fl = Flatten()(d1)
#     preds = Dense(1, activation='sigmoid')(fl)
#     model = Model(inputs=inp, outputs=preds)
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model = Sequential()
    model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

logger = Logger(patience=10, out_path=save_directory, out_fn='cv_{}.h5')
#nn_params = {'nb_epoch':32, 'batch_size':256, 'callbacks':[logger], 'verbose':1}
nn_params = {'epochs':32, 'batch_size':256, 'verbose':1}

# Wrappers

In [0]:
class WrapLGB(LGBMClassifier):
    """This is template for user-defined class wrapper.
    Use this template to pass any ``fit`` and ``predict`` arguments.
    """
    def fit(self, X, y):
        X_tr, X_val, y_tr, y_val = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)
        return super(WrapLGB, self).fit(X_tr, y_tr, 
                                        early_stopping_rounds=5, 
                                        eval_set=[(X_val, y_val)], 
                                        eval_metric='auc', verbose=1)

    def predict(self, X):
        return super(WrapLGB, self).predict(X, 
               num_iteration=self.best_iteration_)
    
    
class WrapXGB(XGBClassifier):
    """This is template for user-defined class wrapper.
    Use this template to pass any ``fit`` and ``predict`` arguments.
    """
    def fit(self, X, y):
        X_tr, X_val, y_tr, y_val = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)
        return super(WrapXGB, self).fit(X_tr, y_tr, 
                                        early_stopping_rounds=5, 
                                        eval_set=[(X_val, y_val)], 
                                        eval_metric='auc', verbose=1)

    def predict(self, X):
        return super(WrapXGB, self).predict(X, 
               num_iteration=self.best_iteration_)
    
    
class WrapCB(CatBoostClassifier):
    """This is template for user-defined class wrapper.
    Use this template to pass any ``fit`` and ``predict`` arguments.
    """
    def fit(self, X, y):
        X_tr, X_val, y_tr, y_val = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)
        return super(WrapCB, self).fit(X_tr, y_tr, 
                                        early_stopping_rounds=5, 
                                        eval_set=[(X_val, y_val)], verbose=1)

    def predict(self, X):
        return super(WrapCB, self).predict(X, 
               num_iteration=self.best_iteration_)
    
    
class WrapDNN(KerasClassifier):
    """This is template for user-defined class wrapper.
    Use this template to pass any ``fit`` and ``predict`` arguments.
    """
    def fit(self, X, y):
        X_tr, X_val, y_tr, y_val = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)
        return super(WrapDNN, self).fit(X_tr, y_tr, 
                                        early_stopping_rounds=5, 
                                        eval_set=[(X_val, y_val)], verbose=1)

    def predict(self, X):
        return super(WrapDNN, self).predict(X, 
               num_iteration=self.best_iteration_)

# Stacking Pipeline

In [12]:
# Specify steps of Pipeline

# Same paramaters to models, but different subfeatures by:
# 1. Select a random number for the number of features to select
# 2. Randomly select that many features
num_subsets = 5
subsets = []
features = list(range(200))
number = list(range(1, 201))
pipe_models_1 = []
for num in range(num_subsets):
    num_features = np.random.choice(number, size=1)[0]
    subset = np.random.choice(features, size=num_features, replace=False, p=None)
    subsets.append(subset)
    xgb_params_rand = random.choice(xgbp_list)
    lgb_params_rand = random.choice(lgbp_list)
    catboost_params_rand = random.choice(cbp_list)
    print("xgb params random chosen: ", xgb_params_rand)
    print("lgb params random chosen: ", lgb_params_rand)
    print("catboost params random chosen: ", catboost_params_rand)

    cl1 = ('gnb' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), GaussianNB()))
#     cl2 = ('xgb' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), WrapXGB(**xgb_params)))
#     cl3 = ('lgbm' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), WrapLGB(**lgb_params)))
#     cl4 = ('cb' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), WrapCB(**catboost_params)))
    cl2 = ('xgb' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), WrapXGB(**xgb_params_rand)))
    cl3 = ('lgbm' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), WrapLGB(**lgb_params_rand)))
    cl4 = ('cb' + str(num), make_pipeline(ColumnSelector(cols=tuple(subset)), WrapCB(**catboost_params_rand)))
    pipe_models_1.append(cl1)
    pipe_models_1.append(cl2)
    pipe_models_1.append(cl3)
    pipe_models_1.append(cl4)
print("Number of level 1 models: ", len(pipe_models_1))
print("Number of subsets: ", len(subsets))
print("Subsets: ", subsets)
# pipe_models_1 = [ 
#     ('gnb', GaussianNB()),
#     #('lg', LogisticRegression(random_state=0)),
#     #('nn', KerasClassifier(build_fn=dnn, epochs=2, batch_size=32, verbose=0, callbacks=[logger])),
#     #('nn', KerasClassifier(build_fn=dnn, epochs=2, batch_size=32, verbose=0)),
#     #('etc', ExtraTreesClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
#     #('rfc', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
#     ('xgb', XGBClassifier(**xgb_params)),           
#     ('lgbm', LGBMClassifier(**lgb_params)),
#     ('cb', CatBoostClassifier(**catboost_params))
# ]
subset = list(range(0, len(pipe_models_1), 2))
#subset = list(range(0, len(pipe_models_1)))
xgb_params_rand = random.choice(xgbp_list)
lgb_params_rand = random.choice(lgbp_list)
cb_params_rand = random.choice(cbp_list)
print("xgb params random chosen: ", xgb_params_rand)
print("lgb params random chosen: ", lgb_params_rand)
print("catboost params random chosen: ", catboost_params_rand)
pipe_models_2 = [ 
    #('gnb', GaussianNB()),
    #('nn', KerasClassifier(build_fn=dnn, epochs=2, batch_size=32, verbose=1)),
    #('etc', ExtraTreesClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
    #('rfc', RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3)),
#     ('xgb', make_pipeline(ColumnSelector(cols=tuple(subset)), WrapXGB(**xgb_params))),           
#     ('lgbm', make_pipeline(ColumnSelector(cols=tuple(subset)), WrapLGB(**lgb_params))),
#     ('cb', make_pipeline(ColumnSelector(cols=tuple(subset)), WrapCB(**catboost_params)))
    ('xgb', make_pipeline(ColumnSelector(cols=tuple(subset)), WrapXGB(**xgb_params_rand))),           
    ('lgbm', make_pipeline(ColumnSelector(cols=tuple(subset)), WrapLGB(**lgb_params_rand))),
    ('cb', make_pipeline(ColumnSelector(cols=tuple(subset)), WrapCB(**catboost_params_rand)))
]

stack1 = StackingTransformer(pipe_models_1,                   # list of models
                               #X_train, y_train, X_test,   # data
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               #mode='oof_pred',            # mode: oof for train set, fit on full 
                               variant='A',                            #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               ##save_dir='.',               # save result and log in current dir 
                               #save_dir=save_directory,                                 #     (to disable saving - set to None)
                               metric=auc,            # metric: callable
                               n_folds=5,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=1)                  # print all info

stack2 = StackingTransformer(pipe_models_2,                   # list of models
                               #X_train, y_train, X_test,   # data
                               regression=False,           # classification task (if you need 
                                                           #     regression - set to True)
                               #mode='oof_pred',            # mode: oof for train set, fit on full 
                               variant='A',                            #     train and predict test set once
                               needs_proba=True,           # predict probabilities (if you need 
                                                           #     class labels - set to False) 
                               ##save_dir='.',               # save result and log in current dir 
                               #save_dir=save_directory,                                 #     (to disable saving - set to None)
                               metric=auc,            # metric: callable
                               n_folds=5,                  # number of folds
                               stratified=True,            # stratified split for folds
                               shuffle=True,               # shuffle the data
                               random_state=0,             # ensure reproducibility
                               verbose=1)                  # print all info

xgb params random chosen:  {'tree_method': 'hist', 'objective': 'binary:logistic', 'eval_metric': 'auc', 'learning_rate': 0.0936165921314771, 'max_depth': 2, 'colsample_bytree': 0.3561271102144279, 'subsample': 0.8246604621518232, 'min_child_weight': 53, 'gamma': 9.943467991283027, 'silent': 1}
lgb params random chosen:  {'num_leaves': 31, 'min_data_in_leaf': 30, 'objective': 'binary', 'max_depth': -1, 'learning_rate': 0.05, 'min_child_samples': 20, 'boosting': 'gbdt', 'feature_fraction': 0.9, 'bagging_freq': 1, 'bagging_fraction': 0.9, 'bagging_seed': 11, 'metric': 'auc', 'lambda_l1': 0.1, 'verbosity': -1, 'nthread': 4, 'random_state': 2019}
catboost params random chosen:  {'subsample': 0.36, 'random_strength': 0, 'max_depth': 3, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'learning_rate': 0.02, 'iterations': 60000, 'bootstrap_type': 'Bernoulli', 'l2_leaf_reg': 0.3, 'random_seed': 432013, 'od_type': 'Iter', 'border_count': 128}
xgb params random chosen:  {'max_depth': 2, 'eval_m

In [0]:
from scipy.stats import rankdata
from sklearn.base import BaseEstimator, TransformerMixin
class Rank(BaseEstimator, TransformerMixin):
    """Define fit and transform for sklearn pipeline"""
    def __init__(self, method='average'):
        self.method = method
    def fit(self, X, y):
        return self
    def transform(self, X):
        return np.apply_along_axis(rankdata, 0, X)
        #return X
stack3 = make_pipeline(Rank(), XGBClassifier(**xgb_params))
#stack3 = make_pipeline(Rank(), WrapDNN(**nn_params))

steps = [('stack1', stack1),
         ('stack2', stack2),
        ('stack3', stack3)]

# Init Pipeline
pipe = Pipeline(steps)

In [0]:
# Fit
pipe = pipe.fit(X_train, y_train)

task:         [classification]
n_classes:    [2]
metric:       [auc]
variant:      [A]
n_estimators: [20]

estimator  0: [gnb0: Pipeline]
    MEAN:     [0.75889022] + [0.01277707]

estimator  1: [xgb0: Pipeline]
[0]	validation_0-auc:0.5
Will train until validation_0-auc hasn't improved in 5 rounds.
[1]	validation_0-auc:0.5
[2]	validation_0-auc:0.528498
[3]	validation_0-auc:0.549688
[4]	validation_0-auc:0.548669
[5]	validation_0-auc:0.549125
[6]	validation_0-auc:0.547689
[7]	validation_0-auc:0.553408
[8]	validation_0-auc:0.58132
[9]	validation_0-auc:0.581404
[10]	validation_0-auc:0.58163
[11]	validation_0-auc:0.586052
[12]	validation_0-auc:0.58459
[13]	validation_0-auc:0.597972
[14]	validation_0-auc:0.594605
[15]	validation_0-auc:0.601148
[16]	validation_0-auc:0.601439
[17]	validation_0-auc:0.60928
[18]	validation_0-auc:0.622261
[19]	validation_0-auc:0.632728
[20]	validation_0-auc:0.637351
[21]	validation_0-auc:0.636238
[22]	validation_0-auc:0.64055
[23]	validation_0-auc:0.640078
[24]	v

# roc on validation data

In [0]:
y_pred = pipe.predict_proba(X_test)
from sklearn.metrics import roc_auc_score

# Final prediction score
# print('Final prediction score: %.8f' % log_loss(y_test, y_pred))
y_pred_final = [elem[1] for elem in y_pred]
roc_auc_score(y_test, y_pred_final)

Transforming...

estimator  0: [gnb0: Pipeline]
    DONE

estimator  1: [xgb0: Pipeline]
    DONE

estimator  2: [lgbm0: Pipeline]
    DONE

estimator  3: [cb0: Pipeline]
    DONE

estimator  4: [gnb1: Pipeline]
    DONE

estimator  5: [xgb1: Pipeline]
    DONE

estimator  6: [lgbm1: Pipeline]
    DONE

estimator  7: [cb1: Pipeline]
    DONE

estimator  8: [gnb2: Pipeline]
    DONE

estimator  9: [xgb2: Pipeline]
    DONE

estimator 10: [lgbm2: Pipeline]
    DONE

estimator 11: [cb2: Pipeline]
    DONE

estimator 12: [gnb3: Pipeline]
    DONE

estimator 13: [xgb3: Pipeline]
    DONE

estimator 14: [lgbm3: Pipeline]
    DONE

estimator 15: [cb3: Pipeline]
    DONE

estimator 16: [gnb4: Pipeline]
    DONE

estimator 17: [xgb4: Pipeline]
    DONE

estimator 18: [lgbm4: Pipeline]
    DONE

estimator 19: [cb4: Pipeline]
    DONE

Transforming...

estimator  0: [xgb: Pipeline]
    DONE

estimator  1: [lgbm: Pipeline]
    DONE

estimator  2: [cb: Pipeline]
    DONE



0.7500316633443099

# rocauc on training data

In [0]:
y_pred = pipe.predict_proba(train_data_x)
y_pred_final = [elem[1] for elem in y_pred]
roc_auc_score(train_data_y, y_pred_final)

Transforming...

estimator  0: [gnb0: Pipeline]
    DONE

estimator  1: [xgb0: Pipeline]
    DONE

estimator  2: [lgbm0: Pipeline]
    DONE

estimator  3: [cb0: Pipeline]
    DONE

estimator  4: [gnb1: Pipeline]
    DONE

estimator  5: [xgb1: Pipeline]
    DONE

estimator  6: [lgbm1: Pipeline]
    DONE

estimator  7: [cb1: Pipeline]
    DONE

estimator  8: [gnb2: Pipeline]
    DONE

estimator  9: [xgb2: Pipeline]
    DONE

estimator 10: [lgbm2: Pipeline]
    DONE

estimator 11: [cb2: Pipeline]
    DONE

estimator 12: [gnb3: Pipeline]
    DONE

estimator 13: [xgb3: Pipeline]
    DONE

estimator 14: [lgbm3: Pipeline]
    DONE

estimator 15: [cb3: Pipeline]
    DONE

estimator 16: [gnb4: Pipeline]
    DONE

estimator 17: [xgb4: Pipeline]
    DONE

estimator 18: [lgbm4: Pipeline]
    DONE

estimator 19: [cb4: Pipeline]
    DONE

Transforming...

estimator  0: [xgb: Pipeline]
    DONE

estimator  1: [lgbm: Pipeline]
    DONE

estimator  2: [cb: Pipeline]
    DONE



0.6254165028431977

In [0]:
# Save Pipeline
import joblib
_ = joblib.dump(pipe, save_directory + 'pipe_with_stack.pkl')
# Load Pipeline
pipe_loaded = joblib.load(save_directory + 'pipe_with_stack.pkl')

# Predict on Kaggle Test

In [0]:
kaggle_pred = pipe.predict_proba(test_data_x)

In [0]:
output_df = pd.DataFrame()
output_df["ID_code"] = test_data_df["ID_code"]
pred_final = [elem[1] for elem in kaggle_pred]
output_df["target"] = pred_final
output_df.to_csv(save_directory + "predictions.csv", index=False)